In [1]:
import pycountry
import pandas as pd
import numpy as np
import re
import os

In [2]:
df = pd.read_csv('data/COSMOS_Requests.csv')

In [3]:
# Null rows are deleted from 'Email' column
df = df[~df["Email"].isnull()]

In [4]:
df

,First name,Last name,Email,University,Date,Counry,Webpage,Country,Domain
0,Alejandro,Martin,a00442746@itesm.mx,Tecnologico de Monterrey,NaN,Mexico,http://www.itesm.mx/,Mexico,itesm.mx
1,Myrtle-may,Lambert,s1240065@sms.ed.ac.uk,Edinburgh,NaN,UK,NaN,NaN,NaN
2,Katrin,Weller,katrin.weller@gesis.org,GESIS Liebniz,NaN,Germany,NaN,NaN,NaN
3,Muhammad Haroon Shakeel,NaN,mhfateen@gmail.com,Lahore University of Management Sciences Pakistan,NaN,NaN,NaN,NaN,NaN
4,Andrew,Brindle,andrewbr@mail.sju.edu.tw,"St. John's University, Taiwan",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3163,A,W,vozart@mail.ru,St. Petersburg State University,"Feb 8, 2021, 3:54 PM",NaN,NaN,NaN,NaN
3164,Jessica,Roberts,jessicaroberts@ucp.pt,Catholic University of Portugal,"Feb 8, 2021, 3:54 PM",NaN,NaN,NaN,NaN
3165,Xiaowei,Yang,xiaowei.yang@postgrad.manchester.ac.uk\r\n,University of Manchester,"Feb 8, 2021, 5:59 PM",NaN,NaN,NaN,NaN
3166,Pere,Masip,peremm@blanquerna.url.edu,University Ramon Llull,"Feb 8, 2021, 5:59 PM",NaN,NaN,NaN,NaN


In [5]:
# regex patterns
df_country_patterns = pd.read_csv("data/country_regex_patterns.csv")

In [6]:
def match_country_pattern(email, df_country_patterns):
    '''`function returns a country name when it is matched with the regex pattern. '''
    for index, row in df_country_patterns.iterrows():
        if re.search(row['regex_pattern'], email):
            print("theres a match")
            return(row['country_name'])
            break

In [7]:
# Create 'country_name' for countries, we found from emails.
df["country_name"] = df["Email"].apply(lambda x: match_country_pattern(x, df_country_patterns))

theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a m

theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a m

theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a m

theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a match
theres a m

In [9]:
# Create a csv file for null rows country_names column has.
# This file was created to figure out how many countries was written in the spreadsheet 
# before we determined the regex pattern from emails.
df[df["country_name"].isnull()].to_csv("data/temp.csv")

In [10]:
# Drop duplicated Emails
df_missing_countries = pd.read_csv("data/temp.csv").drop_duplicates(subset= "Email",
                                                             ignore_index=True)

In [11]:
# Select only two columns
df_missing_countries = df_missing_countries[["Email", "Country"]]

In [12]:
df_missing_countries

,Email,Country
0,katrin.weller@gesis.org,NaN
1,mhfateen@gmail.com,NaN
2,ha37alves@gmail.com,NaN
3,luisyepez13@gmail.com,NaN
4,apikdriya@yahoo.com,NaN
...,...,...
657,jcaraballo@cscbroward.org,NaN
658,rmpubani@acme-ug.org,NaN
659,arthurs@cardiff.ac.uk\r\n,NaN
660,abbey.lantinberg@sjcds.net,NaN


In [13]:
# Joined two df on email key column.
temp_joined_df = df.merge(df_missing_countries, on='Email', how='left')

In [14]:
# Combine two columns (country_name and country_y) into one (country_name_new)
temp_joined_df['country_name_new'] = temp_joined_df["country_name"].combine_first(temp_joined_df["Country_y"])

In [15]:
# Dropped unnecessary columns from dataframe
dropped_temp_joined_df=temp_joined_df.drop(columns=['Counry', 'Country_x', 'Domain'])

In [46]:
dropped_temp_joined_df

,First name,Last name,Email,University,Date,Webpage,country_name,Country_y,country_name_new,country_iso3
0,Alejandro,Martin,a00442746@itesm.mx,Tecnologico de Monterrey,NaN,http://www.itesm.mx/,Mexico,NaN,Mexico,MEX
1,Myrtle-may,Lambert,s1240065@sms.ed.ac.uk,Edinburgh,NaN,NaN,United Kingdom,NaN,United Kingdom,GBR
2,Katrin,Weller,katrin.weller@gesis.org,GESIS Liebniz,NaN,NaN,None,NaN,NaN,Other
3,Muhammad Haroon Shakeel,NaN,mhfateen@gmail.com,Lahore University of Management Sciences Pakistan,NaN,NaN,None,NaN,NaN,Other
4,Andrew,Brindle,andrewbr@mail.sju.edu.tw,"St. John's University, Taiwan",NaN,NaN,"Taiwan, Province of China",NaN,"Taiwan, Province of China",TWN
...,...,...,...,...,...,...,...,...,...,...
3162,A,W,vozart@mail.ru,St. Petersburg State University,"Feb 8, 2021, 3:54 PM",NaN,Russian Federation,NaN,Russian Federation,RUS
3163,Jessica,Roberts,jessicaroberts@ucp.pt,Catholic University of Portugal,"Feb 8, 2021, 3:54 PM",NaN,Portugal,NaN,Portugal,PRT
3164,Xiaowei,Yang,xiaowei.yang@postgrad.manchester.ac.uk\r\n,University of Manchester,"Feb 8, 2021, 5:59 PM",NaN,None,NaN,NaN,Other
3165,Pere,Masip,peremm@blanquerna.url.edu,University Ramon Llull,"Feb 8, 2021, 5:59 PM",NaN,United States,NaN,United States,USA


In [16]:
def convert_country_iso3(country):
    """converts country name to country codes"""
    try:
        iso3 = pycountry.countries.get(name=country).alpha_3
    except AttributeError:
        iso3 = "Other"
    return iso3

In [17]:
dropped_temp_joined_df.loc[dropped_temp_joined_df["country_name_new"].isnull(),'country_name_new'] = 'NaN'

In [19]:
# Add iso3 column for graph.
dropped_temp_joined_df["country_iso3"] = dropped_temp_joined_df["country_name_new"].apply(lambda x:convert_country_iso3(x))

In [23]:
dropped_temp_joined_df

,First name,Last name,Email,University,Date,Webpage,country_name,Country_y,country_name_new,country_iso3
0,Alejandro,Martin,a00442746@itesm.mx,Tecnologico de Monterrey,NaN,http://www.itesm.mx/,Mexico,NaN,Mexico,MEX
1,Myrtle-may,Lambert,s1240065@sms.ed.ac.uk,Edinburgh,NaN,NaN,United Kingdom,NaN,United Kingdom,GBR
2,Katrin,Weller,katrin.weller@gesis.org,GESIS Liebniz,NaN,NaN,None,NaN,NaN,Other
3,Muhammad Haroon Shakeel,NaN,mhfateen@gmail.com,Lahore University of Management Sciences Pakistan,NaN,NaN,None,NaN,NaN,Other
4,Andrew,Brindle,andrewbr@mail.sju.edu.tw,"St. John's University, Taiwan",NaN,NaN,"Taiwan, Province of China",NaN,"Taiwan, Province of China",TWN
...,...,...,...,...,...,...,...,...,...,...
3162,A,W,vozart@mail.ru,St. Petersburg State University,"Feb 8, 2021, 3:54 PM",NaN,Russian Federation,NaN,Russian Federation,RUS
3163,Jessica,Roberts,jessicaroberts@ucp.pt,Catholic University of Portugal,"Feb 8, 2021, 3:54 PM",NaN,Portugal,NaN,Portugal,PRT
3164,Xiaowei,Yang,xiaowei.yang@postgrad.manchester.ac.uk\r\n,University of Manchester,"Feb 8, 2021, 5:59 PM",NaN,None,NaN,NaN,Other
3165,Pere,Masip,peremm@blanquerna.url.edu,University Ramon Llull,"Feb 8, 2021, 5:59 PM",NaN,United States,NaN,United States,USA


In [32]:
geo_df = dropped_temp_joined_df['country_name_new'].value_counts().reset_index()

In [38]:
geo_df["country_iso3"] = geo_df["index"].apply(lambda x:convert_country_iso3(x))

In [42]:
geo_df

,index,country_name_new,country_iso3
0,United Kingdom,1322,GBR
1,NaN,584,Other
2,United States,344,USA
3,Canada,82,CAN
4,Australia,78,AUS
...,...,...,...
63,Costa Rica,1,CRI
64,Romania,1,ROU
65,Qatar,1,QAT
66,Bangladesh,1,BGD


In [43]:
geo_df.columns= ["country", "download_count", "country_iso3"]

In [44]:
geo_df

,country,download_count,country_iso3
0,United Kingdom,1322,GBR
1,NaN,584,Other
2,United States,344,USA
3,Canada,82,CAN
4,Australia,78,AUS
...,...,...,...
63,Costa Rica,1,CRI
64,Romania,1,ROU
65,Qatar,1,QAT
66,Bangladesh,1,BGD


In [45]:
geo_df.to_csv("/Users/serenay/Documents/Research_projects/interactive-geo-map/data/cosmos_download.csv")